# Analyse des scores de risque Signaux Faibles - Création d'indicateurs régionaux
Dans le cadre d'une demande du Ministère du Travail, Signaux Faibles réalise une analyse agrégée aux niveaux géographiques de la région et du département, pour fournir différents indicateurs de risque territorialisés.

Ce notebook vise à charger les données provenant de nos prédictions de risque pour Mars 2020, et à produire des indicateurs agrégés qui seront ultérieurement aposés sur des fonds de cartographie.

In [2]:
%config Completer.use_jedi = False

In [1]:
import os.path
from pymongo import MongoClient
from pymongo.cursor import Cursor

import pandas as pd
import numpy as np
from datetime import datetime
import pytz

# Set logging level to INFO
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from predictsignauxfaibles.utils import MongoDBQuery, MongoParams
import predictsignauxfaibles.config as global_config

import config as cab_config
import utils


# Part 1 - Métriques à mars 2020 (modèle)

In [3]:
load_features_from_file = True
load_scores_from_file = True
features_path = "/home/simon.lebastard/predictsignauxfaibles/data/features_2103.json"
scores_path = "/home/simon.lebastard/predictsignauxfaibles/data/scores_2103.json"
postproc_path = "/home/simon.lebastard/predictsignauxfaibles/data/postproc_2103.json"

## Fetching features and scores data
### Option 1: Fetch data from each dataset

In [5]:
features = utils.load_features(date_min="2020-02-01", date_max="2020-02-28", from_file=False, filepath=features_path)
logging.info(f"Loaded {features.shape[0]} rows and {features.shape[1]} columns")

INFO:root:Loading features from MongoDB...
INFO:root:... Success! Unravelling dataframe...
INFO:root:...done!
INFO:root:Saving fetched Features data to disk
INFO:root:Success
INFO:root:Loaded 956765 rows and 31 columns


In [6]:
scores = utils.load_scores(batch_name="2102_altares", algo_name="mars2021_v0", from_file=True, filepath=scores_path)
logging.info(f"Loaded {scores.shape[0]} rows and {scores.shape[1]} columns")

INFO:root:Succesfully loaded Scores data from /home/simon.lebastard/predictsignauxfaibles/data/scores_2103.json
INFO:root:Loaded 657296 rows and 7 columns


In [7]:
features["periode"] = features.periode.apply(datetime_to_str)
scores["periode"] = scores.periode.apply(datetime_to_str)

In [9]:
df = pd.merge(scores, features, on=['siret', 'periode'], how='inner')

In [11]:
if not os.path.isfile(postproc_path):
    logging.info("Saving joined post-processed data to disk...")
    df.to_json(postproc_path, orient="records", default_handler=str)
    logging.info(f"Saved to {postproc_path}")

INFO:root:Saving joined post-processed data to disk...
INFO:root:Saved to /home/simon.lebastard/predictsignauxfaibles/data/postproc_2103.json


### Option 2: Load data directly from df stored on disk

In [ ]:
if os.path.isfile(postproc_path):
    print("Loading post-processed data to disk")
    df = pd.read_json(postproc_path, orient="records")

## Aggregation of region-wide features

Niveaux de granularité considérés:
- région
- département

Pour chaque niveau de granularité:
- compter le nombre d'établissements flaguées rouge par région
- compter le nombre d'établissements flaguées orange par région
- compter le nombre d'établissements flaguées en rouge OU en orange
- rapporter ces nombre d'établissements au nombre total d'établissements dans la zone géographique
- compter le nombre de défaillances effectives sur une période donnée, et calculer le ratio correspondants

Pour toutes les grandeurs calculées précédemment, calculer des équivalents en nombre d'employés concernés.

Pour les établissements ayant un risque de défaillance modéré ou fort (flagguée en rouge OU en orange), communiquer:
- ratio $\frac{dette_{ouvriere}}{cotisation}$ moyen
- recours moyen à l'activité partielle

On pourra éventuellement ajouter à ces premières métriques:
- des ratios financiers provenant de la DE de la Banque de France
- le nombre de jour moyen de retard de paiement aux fournisseurs (donnée Paydex)

### Preprocessing steps

In [12]:
def preprocess(df):
    # create an outcome flag based only on failures since the beginning of the COVID crisis
    df["failure"] = (df["time_til_failure"]>=0) & (df["time_til_failure"]<12) # todo: automatiser le nombre de mois à regarder vers l'avant: entre mars 2020 et <THIS_MONTH>
    df["failure"] = df.failure.astype(int)

    # encode alert level into integer
    df["alert_flag"] = df.alert.replace({"Pas d'alerte": 0, "Alerte seuil F1": 1, "Alerte seuil F2": 2})
    df["alert_bin"] = (df.alert_flag > 0)

    # ratio dette/cotisation sur la part salariale des cotisations sociales
    df["ratio_dette_ouvriere"] = df["montant_part_ouvriere"] / df["cotisation"]
    df["ratio_dette_patronale"] = df["montant_part_patronale"] / df["cotisation"]
    return df

def replace_nans(df, replace_dct):
    for field, rpl in replace_dct.items():
        df[field].fillna(value=rpl, inplace=True)

In [13]:
replace_nans(df, cab_config.NAN_RPL)
df = preprocess(df)

Vérification des effectifs par catégorie:
- entrée en procédure collective
- flagging par l'algorithme SF
- flagging binaire par l'algorithme SF (True si un établissement est flagué en rouge OU en orange, False si l'établissement est flagué Vert, ie non flagué)

In [14]:
df.groupby(by=["failure"]).siret.count()

failure
0    644012
1     13284
Name: siret, dtype: int64

In [15]:
df.groupby(by=["alert_flag"]).siret.count()

alert_flag
0    591740
1     37598
2     27958
Name: siret, dtype: int64

In [16]:
df.groupby(by=["alert_bin"]).siret.count()

alert_bin
False    591740
True      65556
Name: siret, dtype: int64

### Building aggregation dataframe

In [ ]:
cab_config.FEATURES_LIST

In [17]:
def aggregate_stats(geo_attr, outcome_attr):
    assert outcome_attr in ["alert_flag", "alert_bin", "failure", "outcome"]

    risk_ape3_stats = df.groupby(by=[geo_attr,outcome_attr,"libelle_naf","libelle_ape3"]).agg(
        siret_count=('siret', 'count'),
        effectif_tot=('effectif', 'sum'),
    )
    risk_naf_stats = df.groupby(by=[geo_attr,outcome_attr,"libelle_naf"]).agg(
        siret_count=('siret', 'count'),
        effectif_tot=('effectif', 'sum')
    )
    risk_stats = df.groupby(by=[geo_attr,outcome_attr]).agg(
        siret_count=('siret', 'count'),
        effectif_tot=('effectif', 'sum'),
        ratiodette_ouvr_avg=('ratio_dette_ouvriere', 'mean'),
        ratiodette_patr_avg=('ratio_dette_patronale', 'mean'),
        ratiodette_avg=('ratio_dette', 'mean'),
        apart_autr_avg=('apart_heures_autorisees', 'mean'),
        apart_cons_avg=('apart_heures_consommees', 'mean'),
        apart_cumcons_avg=('apart_heures_consommees_cumulees', 'mean'),
        paydex_avg=('paydex_nb_jours', 'mean'),
        taux_endettement_avg=('taux_endettement', 'mean'),
    )
    national_stats = df.groupby(by=[outcome_attr]).agg(
        siret_count=('siret', 'count'),
        effectif_tot=('effectif', 'sum'),
        ratiodette_ouvr_avg=('ratio_dette_ouvriere', 'mean'),
        ratiodette_patr_avg=('ratio_dette_patronale', 'mean'),
        ratiodette_avg=('ratio_dette', 'mean'),
        apart_autr_avg=('apart_heures_autorisees', 'mean'),
        apart_cons_avg=('apart_heures_consommees', 'mean'),
        apart_cumcons_avg=('apart_heures_consommees_cumulees', 'mean'),
        paydex_avg=('paydex_nb_jours', 'mean'),
        taux_endettement_avg=('taux_endettement', 'mean'),
    )

    risk_stats['siret_rate'] = 100*risk_stats.siret_count / risk_stats.groupby(by=geo_attr).siret_count.sum()
    risk_stats['effectif_rate'] = 100*risk_stats.effectif_tot / risk_stats.groupby(by=geo_attr).effectif_tot.sum()

    risk_stats['ape3_mostatrisk_eff'] = risk_ape3_stats.loc[risk_ape3_stats.index.get_level_values("libelle_naf").isin([LIBELLE_NAF[code] for code in ["B", "C", "D", "E"]])].groupby(by=[geo_attr,outcome_attr]).effectif_tot.idxmax()
    risk_stats['ape3_mostatrisk_eff'] = risk_stats.ape3_mostatrisk_eff.apply(lambda x: x[3] if isinstance(x, tuple) else "None")
    risk_stats['ape3_risk_eff'] = risk_ape3_stats.loc[risk_ape3_stats.index.get_level_values("libelle_naf").isin([LIBELLE_NAF[code] for code in ["B", "C", "D", "E"]])].groupby(by=[geo_attr,outcome_attr]).effectif_tot.max()

    risk_stats['naf_mostatrisk_eff'] = risk_naf_stats.groupby(by=[geo_attr,outcome_attr]).effectif_tot.idxmax()
    risk_stats['naf_mostatrisk_eff'] = risk_stats.naf_mostatrisk_eff.apply(lambda x: x[2] if isinstance(x, tuple) else "None")
    risk_stats['naf_risk_eff'] = risk_naf_stats.groupby(by=[geo_attr,outcome_attr]).effectif_tot.max()

    risk_stats['ape3_mostatrisk_etab'] = risk_ape3_stats.loc[risk_ape3_stats.index.get_level_values("libelle_naf").isin([LIBELLE_NAF[code] for code in ["B", "C", "D", "E"]])].groupby(by=[geo_attr,outcome_attr]).siret_count.idxmax()
    risk_stats['ape3_mostatrisk_etab'] = risk_stats.ape3_mostatrisk_etab.apply(lambda x: x[3] if isinstance(x, tuple) else "None")
    risk_stats['ape3_risk_etab'] = risk_ape3_stats.loc[risk_ape3_stats.index.get_level_values("libelle_naf").isin([LIBELLE_NAF[code] for code in ["B", "C", "D", "E"]])].groupby(by=[geo_attr,outcome_attr]).siret_count.max()

    risk_stats['naf_mostatrisk_etab'] = risk_naf_stats.groupby(by=[geo_attr,outcome_attr]).siret_count.idxmax()
    risk_stats['naf_mostatrisk_etab'] = risk_stats.naf_mostatrisk_etab.apply(lambda x: x[2] if isinstance(x, tuple) else "None")
    risk_stats['naf_risk_etab'] = risk_naf_stats.groupby(by=[geo_attr,outcome_attr]).siret_count.max()

    national_stats['siret_rate'] = 100*national_stats.siret_count / national_stats.siret_count.sum()
    national_stats['effectif_rate'] = 100*national_stats.effectif_tot / national_stats.effectif_tot.sum()


    risk_stats['siret_rate_to_ntl_avg'] = 100*(risk_stats.siret_rate - national_stats.siret_rate) / national_stats.siret_rate
    risk_stats['effectif_rate_to_ntl_avg'] = 100*(risk_stats.effectif_rate - national_stats.effectif_rate) / national_stats.effectif_rate
    risk_stats['ratiodette_ouvr_avg'] = 100*(risk_stats.ratiodette_ouvr_avg - national_stats.ratiodette_ouvr_avg) / national_stats.ratiodette_ouvr_avg
    risk_stats['ratiodette_patr_avg'] = 100*(risk_stats.ratiodette_patr_avg - national_stats.ratiodette_patr_avg) / national_stats.ratiodette_patr_avg
    risk_stats['ratiodette_avg'] = 100*(risk_stats.ratiodette_avg - national_stats.ratiodette_avg) / national_stats.ratiodette_avg
    risk_stats['apart_autr_avg'] = 100*(risk_stats.apart_autr_avg - national_stats.apart_autr_avg) / national_stats.apart_autr_avg
    risk_stats['apart_cons_avg_to_ntl_avg'] = 100*(risk_stats.apart_cons_avg - national_stats.apart_cons_avg) / national_stats.apart_cons_avg
    risk_stats['apart_cumcons_avg_to_ntl_avg'] = 100*(risk_stats.apart_cumcons_avg - national_stats.apart_cumcons_avg) / national_stats.apart_cumcons_avg
    risk_stats['paydex_avg_to_ntl_avg'] = 100*(risk_stats.paydex_avg - national_stats.paydex_avg) / national_stats.paydex_avg
    risk_stats['taux_endettement_avg'] = 100*(risk_stats.taux_endettement_avg - national_stats.taux_endettement_avg) / national_stats.taux_endettement_avg

    return risk_stats

In [18]:
reg_risk_stats = aggregate_stats(geo_attr="region", outcome_attr="alert_flag")
reg_riskbin_stats = aggregate_stats(geo_attr="region", outcome_attr="alert_bin")
reg_fail_stats = aggregate_stats(geo_attr="region", outcome_attr="failure")

dpt_risk_stats = aggregate_stats(geo_attr="departement", outcome_attr="alert_flag")
dpt_riskbin_stats = aggregate_stats(geo_attr="departement", outcome_attr="alert_bin")
dpt_fail_stats = aggregate_stats(geo_attr="departement", outcome_attr="failure")

In [19]:
reg_risk_outpath_root = "/home/simon.lebastard/predictsignauxfaibles/data/reg_2103"
dpt_risk_outpath_root = "/home/simon.lebastard/predictsignauxfaibles/data/dpt_2103"

reg_risk_stats.to_csv(f"{reg_risk_outpath_root}_riskflag.csv")
dpt_risk_stats.to_csv(f"{dpt_risk_outpath_root}_riskflag.csv")
reg_riskbin_stats.to_csv(f"{reg_risk_outpath_root}_riskbin.csv")
dpt_riskbin_stats.to_csv(f"{dpt_risk_outpath_root}_riskbin.csv")
reg_fail_stats.to_csv(f"{reg_risk_outpath_root}_failures.csv")
dpt_fail_stats.to_csv(f"{dpt_risk_outpath_root}_failures.csv")